In [2]:
debug = False
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
import numpy as np
import os
import sys
import time
import datetime
import glob
sys.path.append('../py/')
from s027_kfold_ods import ods_kfold
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import logger_func, get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature, impute_feature
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

#========================================================================
# Args
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term', 'no_out_flg']
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
#========================================================================

2019-02-19 20:42:11,963 utils 400 [INFO]    [logger_func] start 


In [60]:
base = utils.read_pkl_gzip('../input/base_no_out_clf.gz')[[key, target]].set_index(key)
# df_hist = pd.read_csv('../input/historical_transactions.csv')
# df_hist['purchase_amount_new'] = np.round(df_hist['purchase_amount'] / 0.00150265118 + 497.06, 2)

In [46]:
# id_list = utils.read_pkl_gzip('../input/0217_merchant_one_id_list.gz')
# tmp = df_hist.loc[df_hist[key].isin(id_list), :]
tmp = df_hist.copy()

use_cols = [key, 'month_lag', 'purchase_date', 'authorized_flag', 'installments', 'merchant_id', 'purchase_amount_new', 'subsector_id', 'merchant_category_id']
tmp = tmp[use_cols]

# First purchase_date amount
date_min = tmp.groupby(key)['purchase_date'].min().reset_index()
date_min = tmp.merge(date_min, how='inner', on=[key, 'purchase_date'])
date_min = date_min[[key, 'merchant_id', 'purchase_amount_new']].rename(columns={'purchase_amount_new':'purchase_amount_new_first'})

# First purchase_month amount
month_min = tmp.groupby(key)['month_lag'].min().reset_index()
month_min = tmp.merge(month_min, how='inner', on=[key, 'month_lag'])
month_min = month_min[[key, 'merchant_id', 'purchase_amount_new']].groupby([key, 'merchant_id'])['purchase_amount_new'].sum().reset_index().rename(columns={'purchase_amount_new':'purchase_amount_new_first_month'})

# amount per first amount or first month amount
tmp = tmp.merge(date_min, how='inner', on=[key, 'merchant_id']).merge(month_min, how='inner', on=[key, 'merchant_id'])
tmp['purchase_amount_new_per_first_amount'] =  tmp['purchase_amount_new'] / tmp['purchase_amount_new_first']
tmp['purchase_amount_new_per_first_month_amount'] =  tmp['purchase_amount_new'] / tmp['purchase_amount_new_first_month']

tmp['purchase_amount_new_replace'] = tmp['purchase_amount_new'].replace(22.9, 27.9)
mer_amount_nunique = tmp.groupby([key, 'merchant_id'])['purchase_amount_new_replace'].nunique()
amount_nunique = tmp.groupby([key])['purchase_amount_new_replace'].nunique()
mer_amount_nunique.name = 'merchant_amount_new_nunique'
amount_nunique.name = 'amount_new_nunique'
tmp.set_index([key, 'merchant_id'], inplace=True)
tmp = tmp.join(mer_amount_nunique).join(amount_nunique)
tmp.head(10)

month_lag        purchase_date  \
card_id         merchant_id                                       
C_ID_00007093c1 M_ID_69423b34e4        -11  2017-03-08 15:05:29   
                M_ID_69423b34e4        -11  2017-03-11 15:04:31   
                M_ID_69423b34e4        -10  2017-04-19 12:19:08   
                M_ID_69423b34e4         -1  2018-01-08 16:58:13   
                M_ID_69423b34e4        -11  2017-03-11 15:26:58   
                M_ID_69423b34e4         -8  2017-06-01 18:42:38   
                M_ID_69423b34e4        -10  2017-04-19 11:46:58   
                M_ID_69423b34e4         -7  2017-07-04 17:14:03   
                M_ID_69423b34e4        -12  2017-02-14 14:00:43   
                M_ID_69423b34e4         -2  2017-12-09 11:34:09   

                                authorized_flag  installments  \
card_id         merchant_id                                     
C_ID_00007093c1 M_ID_69423b34e4               Y             1   
                M_ID_69423b34e4               Y             1   
                M_ID_69423b34e4               Y             1   
                M_ID_69423b34e4               Y             1   
                M_ID_69423b34e4               Y             1   
                M_ID_69423b34e4               Y             1   
                M_ID_69423b34e4               Y             1   
                M_ID_69423b34e4               Y             1   
                M_ID_69423b34e4               Y             2   
                M_ID_69423b34e4               N             1   

                                 purchase_amount_new  subsector_id  \
card_id         merchant_id                                          
C_ID_00007093c1 M_ID_69423b34e4               230.00            33   
                M_ID_69423b34e4               120.00            33   
                M_ID_69423b34e4               218.45            33   
                M_ID_69423b34e4               200.00            33   
                M_ID_69423b34e4               120.00            33   
                M_ID_69423b34e4               120.55            33   
                M_ID_69423b34e4               500.00            33   
                M_ID_69423b34e4               121.60            33   
                M_ID_69423b34e4               521.80            33   
                M_ID_69423b34e4               400.00            33   

                                 merchant_category_id  \
card_id         merchant_id                             
C_ID_00007093c1 M_ID_69423b34e4                   705   
                M_ID_69423b34e4                   705   
                M_ID_69423b34e4                   705   
                M_ID_69423b34e4                   705   
                M_ID_69423b34e4                   705   
                M_ID_69423b34e4                   705   
                M_ID_69423b34e4                   705   
                M_ID_69423b34e4                   705   
                M_ID_69423b34e4                   705   
                M_ID_69423b34e4                   705   

                                 purchase_amount_new_first  \
card_id         merchant_id                                  
C_ID_00007093c1 M_ID_69423b34e4                      521.8   
                M_ID_69423b34e4                      521.8   
                M_ID_69423b34e4                      521.8   
                M_ID_69423b34e4                      521.8   
                M_ID_69423b34e4                      521.8   
                M_ID_69423b34e4                      521.8   
                M_ID_69423b34e4                      521.8   
                M_ID_69423b34e4                      521.8   
                M_ID_69423b34e4                      521.8   
                M_ID_69423b34e4                      521.8   

                                 purchase_amount_new_first_month  \
card_id         merchant_id                                        
C_ID_00007093c1 M_ID_69423b34e4         

In [ ]:
# tmp = tmp.reset_index().sort_values(by=[key, 'merchant_id', 'purchase_date'])
# tmp = utils.row_number(df=tmp, level=[key, 'merchant_id'])
tmp.groupby(key)['merchant_id'].nunique().sort_values()

In [70]:
#========================================================================
# Ratio first purchase_amount
#========================================================================
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

base = utils.read_pkl_gzip('../input/base_no_out_clf.gz')[[key, target]].set_index(key)

df_rank2 = tmp[tmp['row_no']==2]
aggs = {}
aggs['purchase_amount_new_per_first_amount'] = ['mean', 'max', 'min']
aggs['purchase_amount_new_per_first_month_amount'] = ['mean', 'max', 'min']
new_cols = get_new_columns('second_purchase', aggs)
df_agg = df_rank2.groupby(key).agg(aggs)
df_agg.columns = new_cols
df_agg['second_purchase_purchase_amount_new_per_first_amount_max-min'] = df_agg['second_purchase_purchase_amount_new_per_first_amount_max'] - df_agg['second_purchase_purchase_amount_new_per_first_amount_min']
df_agg['second_purchase_purchase_amount_new_per_first_month_amount_max-min'] = df_agg['second_purchase_purchase_amount_new_per_first_month_amount_max'] - df_agg['second_purchase_purchase_amount_new_per_first_month_amount_min']
display(df_agg.head())

prefix = '257_one'
base = base.join(df_agg).fillna(-1)
utils.elo_save_feature(df_feat=base, prefix=prefix)

,second_purchase_purchase_amount_new_per_first_amount_mean,second_purchase_purchase_amount_new_per_first_amount_max,second_purchase_purchase_amount_new_per_first_amount_min,second_purchase_purchase_amount_new_per_first_month_amount_mean,second_purchase_purchase_amount_new_per_first_month_amount_max,second_purchase_purchase_amount_new_per_first_month_amount_min,second_purchase_purchase_amount_new_per_first_amount_max-min,second_purchase_purchase_amount_new_per_first_month_amount_max-min
card_id,,,,,,,,
C_ID_00007093c1,0.440782,0.440782,0.440782,0.440782,0.440782,0.440782,0.0,0.0
C_ID_0001238066,9.532726,9.532726,9.532726,0.905058,0.905058,0.905058,0.0,0.0
C_ID_0001793786,1.021178,1.021178,1.021178,1.021178,1.021178,1.021178,0.0,0.0
C_ID_000183fdda,1.000000,1.000000,1.000000,0.500000,0.500000,0.500000,0.0,0.0
C_ID_00024e244b,0.888889,0.888889,0.888889,0.888889,0.888889,0.888889,0.0,0.0


In [71]:
#========================================================================
# Ratio first purchase_amount
#========================================================================
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

base = utils.read_pkl_gzip('../input/base_no_out_clf.gz')[[key, target]].set_index(key)

aggs = {}
aggs['merchant_amount_new_nunique'] = ['mean', 'max', 'min']
aggs['amount_new_nunique'] = ['mean']
new_cols = get_new_columns('variety', aggs)
df_agg = tmp.groupby(key).agg(aggs)
df_agg.columns = new_cols
display(df_agg.head())

prefix = '257_one'
base = base.join(df_agg).fillna(-1)
utils.elo_save_feature(df_feat=base, prefix=prefix)

,variety_merchant_amount_new_nunique_mean,variety_merchant_amount_new_nunique_max,variety_merchant_amount_new_nunique_min,variety_amount_new_nunique_mean
card_id,,,,
C_ID_00007093c1,10.0,10,10,10
C_ID_0001238066,24.0,24,24,24
C_ID_0001506ef0,1.0,1,1,1
C_ID_0001793786,10.0,10,10,10
C_ID_000183fdda,1.0,1,1,1


In [72]:
#========================================================================
# 同じ日、月に同じmerchantでauth0が発生してるか
# 同じmerchantを連続何ヶ月使ってるか
#========================================================================
tmp.head(30)

,card_id,merchant_id,month_lag,purchase_date,authorized_flag,installments,purchase_amount_new,subsector_id,merchant_category_id,purchase_amount_new_first,purchase_amount_new_first_month,purchase_amount_new_per_first_amount,purchase_amount_new_per_first_month_amount,purchase_amount_new_replace,merchant_amount_new_nunique,amount_new_nunique,row_no
0,C_ID_00007093c1,M_ID_69423b34e4,-12,2017-02-14 14:00:43,Y,2,521.80,33,705,521.80,521.80,1.000000,1.000000,521.80,10,10,1
1,C_ID_00007093c1,M_ID_69423b34e4,-11,2017-03-08 15:05:29,Y,1,230.00,33,705,521.80,521.80,0.440782,0.440782,230.00,10,10,2
2,C_ID_00007093c1,M_ID_69423b34e4,-11,2017-03-11 15:04:31,Y,1,120.00,33,705,521.80,521.80,0.229973,0.229973,120.00,10,10,3
3,C_ID_00007093c1,M_ID_69423b34e4,-11,2017-03-11 15:10:15,Y,1,45.90,33,705,521.80,521.80,0.087965,0.087965,45.90,10,10,4
4,C_ID_00007093c1,M_ID_69423b34e4,-11,2017-03-11 15:26:58,Y,1,120.00,33,705,521.80,521.80,0.229973,0.229973,120.00,10,10,5
5,C_ID_00007093c1,M_ID_69423b34e4,-10,2017-04-19 11:46:58,Y,1,500.00,33,705,521.80,521.80,0.958222,0.958222,500.00,10,10,6
6,C_ID_00007093c1,M_ID_69423b34e4,-10,2017-04-19 12:19:08,Y,1,218.45,33,705,521.80,521.80,0.418647,0.418647,218.45,10,10,7
7,C_ID_00007093c1,M_ID_69423b34e4,-8,2017-06-01 18:42:38,Y,1,120.55,33,705,521.80,521.80,0.231027,0.231027,120.55,10,10,8
8,C_ID_00007093c1,M_ID_69423b34e4,-7,2017-07-04 17:14:03,Y,1,121.60,33,705,521.80,521.80,0.233039,0.233039,121.60,10,10,9
9,C_ID_00007093c1,M_ID_69423b34e4,-2,2017-12-09 11:34:09,N,1,400.00,33,705,521.80,521.80,0.766577,0.766577,400.00,10,10,10
